# 基于官方baseline修改，acc94左右

### 本baseline采用pytorch框架，应用ModelArts的Notebook进行开发

### 数据集获取
将您OBS桶中的数据文件加载到此notebook中，将如下代码中"obs-aifood-baseline"修改成您OBS桶名称。

In [1]:
import moxing as mox
# 导入数据
mox.file.copy_parallel('s3://obs-aifood-bj4/aifood','./aifood/')

# 为了方便使用预训练模型，这里已经手动下载了resnet50预训练模型存在自己的OBS桶里
# 下载地址：https://download.pytorch.org/models/resnet50-19c8e357.pth
mox.file.copy_parallel('s3://c4ai/premodel/resnet50-19c8e357.pth',
                       './resnet50-19c8e357.pth')
print("done")

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2
INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:pid: None.	1000/5001
INFO:root:pid: None.	2000/5001
INFO:root:pid: None.	3000/5001
INFO:root:pid: None.	4000/5001
INFO:root:pid: None.	5000/5001


done


In [2]:
!ls 

aifood	resnet50-19c8e357.pth


### 加载依赖

In [3]:

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os


### 加载数据集，并将其分为训练集和测试集

In [4]:
#使用imagenet的均值方差
mean,std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]

# 使用了图像增强，这里加了翻转仿射变换，这里可能会导致验证集的图片也被增强
# 我在推理文件中也加了数据增强，不确定是否有影响，推理文件看最后两个代码块
dataTrans = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
 
    # image data path
data_dir = './aifood/images'
all_image_datasets = datasets.ImageFolder(data_dir, dataTrans)
print(all_image_datasets.class_to_idx) 
# 0.1作为验证集
trainsize = int(0.9*len(all_image_datasets))
testsize = len(all_image_datasets) - trainsize
train_dataset, test_dataset = torch.utils.data.random_split(all_image_datasets,[trainsize,testsize])
   
image_datasets = {'train':train_dataset,'val':test_dataset}
    

    # wrap your data and label into Tensor

    
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                 batch_size=32,
                                                 shuffle=True,
                                                 num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

    # use gpu or not
use_gpu = torch.cuda.is_available()

print(dataset_sizes)

{'三明治': 0, '冰激凌': 1, '土豆泥': 2, '小米粥': 3, '松鼠鱼': 4, '烤冷面': 5, '玉米饼': 6, '甜甜圈': 7, '芒果班戟': 8, '鸡蛋布丁': 9}
{'train': 4500, 'val': 500}


In [5]:
def train_model(model, lossfunc, optimizer, scheduler, num_epochs=10):
    start_time = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0

            # Iterate over data.
            for data in dataloders[phase]:
                # get the inputs
                inputs, labels = data
                

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = lossfunc(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data
                running_corrects += torch.sum(preds == labels.data).to(torch.float32)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    elapsed_time = time.time() - start_time
    print('Training complete in {:.0f}m {:.0f}s'.format(
        elapsed_time // 60, elapsed_time % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
  
    return model

### 模型训练
采用resnet50神经网络结构训练模型,模型训练需要一定时间，等待该段代码运行完成后再往下执行。

In [6]:
# get model and replace the original fc layer with your fc layer

model_ft = models.resnet50(pretrained=False)
# 加载已经下载的预训练模型
model_ft.load_state_dict(torch.load('resnet50-19c8e357.pth'))

num_ftrs = model_ft.fc.in_features

# 加入了dropout，避免过拟合
model_ft.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 10))

# model_ft.fc = nn.Linear(num_ftrs, 10)

if use_gpu:
    model_ft = model_ft.cuda()

    # define loss function
lossfunc = nn.CrossEntropyLoss()

# setting optimizer and trainable parameters

# 这里直接训练所有层的参数
params = list(model_ft.parameters())

# 加了L2正则化，防止过拟合
optimizer_ft = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
# optimizer_ft = optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.0005)


# Decay LR by a factor of 0.1 every 4 epochs
# 每4个epch减小学习率，我是这么理解的
    
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=4, gamma=0.1)

model_ft = train_model(model=model_ft,
                           lossfunc=lossfunc,
                           optimizer=optimizer_ft,
                           scheduler=exp_lr_scheduler,
                           num_epochs=10)

Epoch 0/9
----------
train Loss: 0.0347 Acc: 0.6609
val Loss: 0.0124 Acc: 0.8900
Epoch 1/9
----------
train Loss: 0.0112 Acc: 0.8913
val Loss: 0.0086 Acc: 0.9200
Epoch 2/9
----------
train Loss: 0.0076 Acc: 0.9260
val Loss: 0.0086 Acc: 0.9240
Epoch 3/9
----------
train Loss: 0.0057 Acc: 0.9462
val Loss: 0.0064 Acc: 0.9320
Epoch 4/9
----------
train Loss: 0.0035 Acc: 0.9678
val Loss: 0.0061 Acc: 0.9340
Epoch 5/9
----------
train Loss: 0.0037 Acc: 0.9629
val Loss: 0.0066 Acc: 0.9360
Epoch 6/9
----------
train Loss: 0.0034 Acc: 0.9693
val Loss: 0.0064 Acc: 0.9380
Epoch 7/9
----------
train Loss: 0.0035 Acc: 0.9667
val Loss: 0.0057 Acc: 0.9400
Epoch 8/9
----------
train Loss: 0.0033 Acc: 0.9731
val Loss: 0.0059 Acc: 0.9440
Epoch 9/9
----------
train Loss: 0.0033 Acc: 0.9687
val Loss: 0.0062 Acc: 0.9440
Training complete in 4m 25s
Best val Acc: 0.944000


将训练好的模型保存下来。

In [9]:
torch.save(model_ft.state_dict(), './model.pth')

### 将训练好的模型保存至OBS
将模型保存到OBS桶中model文件夹下，为后续推理测试、模型提交做准备。将如下代码中"obs-aifood-baseline"修改成您OBS桶的名称。


In [10]:
import moxing as mox
# 将c4ai替换成自己的桶名称，
mox.file.copy('./model.pth','s3://c4ai/model_output_re50/model/resnet-50.pth')
print("done")

done


In [ ]:
# 下面是推理文件的改动部分：
# 1数据处理部分,这里不确定必须要改，大佬们可以自行尝试
# 2网络结构部分


In [ ]:
# 代码位置在25行左右

# 这里均值方差使用数据处理部分使用一致
mean,std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
infer_transformation = transforms.Compose([
    transforms.Resize(256),
#     transforms.Resize((224,224)),
    transforms.CenterCrop(224),
    

    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05)),

    
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [ ]:
# 代码位置在158行左右

def resnet50(model_path, **kwargs):

    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = models.resnet50(pretrained=False)
    num_ftrs = model.fc.in_features
    

    model.fc = nn.Sequential(nn.Dropout(0.5),nn.Linear(num_ftrs, 10))


#     model.fc = nn.Linear(num_ftrs, 10)
    model.load_state_dict(torch.load(model_path,map_location ='cpu'))
    # model.load_state_dict(torch.load(model_path))

    model.eval()

    return model